# Project: Hypothesis Testing for Microtransactions
Brian is a Product Manager at FarmBurg, a company that makes a farming simulation social network game.  In the FarmBurg game, you can plow, plant, and harvest different crops.

Today, you will be acting as Brian's data analyst for an A/B Test that he has been conducting.

## Part 1: Testing for Significant Difference

Start by importing the following modules that you'll need for this project:
- `pandas` as `pd`

In [1]:
import pandas as pd

Brian tells you that he ran an A/B test with three different groups: A, B, and C.  You're kind of busy today, so you don't ask too many questions about the differences between A, B, and C.  Maybe they were shown three different versions of an ad.  Who cares?

(HINT: you will care later)

Brian gives you a CSV of results called `clicks.csv`.  It has the following columns:
- `user_id`: a unique id for each visitor to the FarmBerg site
- `ab_test_group`: either `A`, `B`, or `C` depending on which group the visitor was assigned to
- `click_day`: only filled in *if* the user clicked on a link to purchase

Load `clicks.csv` into the variable `df`.

In [2]:
df = pd.read_csv("clicks.csv")
print (df)

                                   user_id group  click_day
0     8e27bf9a-5b6e-41ed-801a-a59979c0ca98     A        NaN
1     eb89e6f0-e682-4f79-99b1-161cc1c096f1     A        NaN
2     7119106a-7a95-417b-8c4c-092c12ee5ef7     A        NaN
3     e53781ff-ff7a-4fcd-af1a-adba02b2b954     A        NaN
4     02d48cf1-1ae6-40b3-9d8b-8208884a0904     A   Saturday
5     5a3ca2d6-25d5-4909-8f07-519f71ee55e8     A        NaN
6     6b929341-1336-4c34-965b-92e368ab160b     A        NaN
7     90b0a07b-e20e-4e0a-872e-5cc303c5676b     A        NaN
8     4b16c922-b2ab-48a8-885c-713ebf0ae159     A        NaN
9     5eb5fc03-fbda-4149-b909-4f5fbc6b152f     A        NaN
10    389ff492-4635-4535-8e42-685f771fccb1     A        NaN
11    1d25885b-56c7-4fdd-bee8-6348c1386bf0     A   Thursday
12    4546807b-8211-4e7b-94cf-4f9c879e284b     A        NaN
13    c447f7f4-a419-41c6-af31-0a17f7943816     A        NaN
14    e75754d9-9342-4f1f-84e1-75c340ef78b2     A        NaN
15    0d38042c-2296-4526-b21c-6b00805688

Define a new column called `is_purchase` which is `Purchase` if `click_day` is not `None` and `No Purchase` if `click_day` is `None`.  This will tell us if each visitor clicked on the Purchase link.

In [3]:
#list for new column "is_purchase"
is_purchase = []

#Traverse through column "click day" 
for row in df['click_day']:
    #Testing if row is "nan"
    if (type(row)==float): 
        is_purchase.append("No Purchase")
    else:
        is_purchase.append("Purchase")

#printing list
#print(is_purchase)

df["is_purchase"] = is_purchase

#printing column
print(df["is_purchase"])

0       No Purchase
1       No Purchase
2       No Purchase
3       No Purchase
4          Purchase
5       No Purchase
6       No Purchase
7       No Purchase
8       No Purchase
9       No Purchase
10      No Purchase
11         Purchase
12      No Purchase
13      No Purchase
14      No Purchase
15         Purchase
16      No Purchase
17      No Purchase
18         Purchase
19      No Purchase
20      No Purchase
21         Purchase
22      No Purchase
23      No Purchase
24         Purchase
25      No Purchase
26      No Purchase
27         Purchase
28      No Purchase
29      No Purchase
           ...     
4968    No Purchase
4969    No Purchase
4970    No Purchase
4971    No Purchase
4972    No Purchase
4973    No Purchase
4974    No Purchase
4975    No Purchase
4976    No Purchase
4977    No Purchase
4978    No Purchase
4979    No Purchase
4980    No Purchase
4981    No Purchase
4982    No Purchase
4983    No Purchase
4984    No Purchase
4985    No Purchase
4986    No Purchase


We want to count the number of users who made a purchase from each group.  Use `groupby` to count the number of `Purchase` and `No Purchase` from each `group`.  Save your answer to the variable `purchase_counts`.

**Hint**: Group by `group` and `is_purchase` and the function `count` on the column `user_id`.

In [12]:
purchase_counts = df.groupby(["group", "is_purchase"])['user_id'].count()

This data is *categorical* and there are *more than 2* conditions, so we'll want to use a chi-squared test to see if there is a significant difference between the three conditions.

Start by filling in the contingency table below with the correct values:
```py
contingency = [[groupA_purchases, groupA_not_purchases],
               [groupB_purchases, groupB_not_purchases],
               [groupC_purchases, groupC_not_purchases]]
```

In [13]:
contingency = [[purchase_counts['A']['Purchase'], purchase_counts['A']['No Purchase']],
               [purchase_counts['B']['Purchase'], purchase_counts['B']['No Purchase']],
               [purchase_counts['C']['Purchase'], purchase_counts['C']['No Purchase']]]
print (contingency)

[[316, 1350], [183, 1483], [83, 1583]]


Now import the function `chi2_contingency` from `scipy.stats` and perform the chi-squared test.

Recall that the *p-value* is the second output of `chi2_contingency`.

In [10]:
from  scipy.stats  import chi2_contingency

In [14]:
chi2_contingency(contingency)

(159.41952879874498, 2.4126213546684264e-35, 2, array([[ 194., 1472.],
        [ 194., 1472.],
        [ 194., 1472.]]))

Great! It looks like a significantly greater portion of users from Group A made a purchase.

## Part 2: Testing for Exceeding a Goal

Your day is a little less busy than you expected, so you decide to ask Brian about his test.

**You**: Hey Brian! What was that test you were running anyway?

**Brian**: It was awesome! We are trying to get users to purchase a small FarmBurg upgrade package.  It's called a microtransaction.  We're not sure how much to charge for it, so we tested three different price points: \$0.99, \$1.99, and \$4.99.  It looks like significantly more people bought the upgrade package for \$0.99, so I guess that's what we'll charge.

**You**: Oh no! I should have asked you this before we did that chi-squared test.  I don't think that this was the right test at all.  It's true that more people wanted purchase the upgrade at \$0.99; you probably expected that.  What we really want to know is if each price point allows us to make enough money that we can exceed some target goal.  Brian, how much do you think it cost to build this feature?

**Brian**: Hmm.  I guess that we need to generate a minimum of $1000 per week in order to justify this project.

**You**: We have some work to do!

How many visitors came to the site this week?

Hint: Look at the length of `df`.

In [15]:
num_visitors = len(df)
print (num_visitors)

4998


Let's assume that this is how many visitors we generally get each week.  Given that, calculate the percent of visitors who would need to purchase the upgrade package at each price point (\$0.99, \$1.99, \$4.99) in order to generate \$1000 per week.

In [16]:
# Calculate the number of people who would need to purchase a $0.99 upgrade in order to generate $1000.
# Then divide by the number of people who visit the site each week.
goal_price = 1000

num_people_a = goal_price/.99
percent_visitors_a = num_people_a/num_visitors

print(num_people_a)
print(percent_visitors_a)

1010.1010101010102
0.20210104243717691


In [18]:
# Calculate the number of people who would need to purchase a $1.99 upgrade in order to generate $1000.
# Then divide by the number of people who visit the site each week.

num_people_b = goal_price/1.99
percent_visitors_b = num_people_b/num_visitors

print(num_people_b)
print(percent_visitors_b)

502.51256281407035
0.10054272965467594


In [20]:
# Calculate the number of people who would need to purchase a $1.99 upgrade in order to generate $1000.
# Then divide by the number of people who visit the site each week.

num_people_c = goal_price/4.99
percent_visitors_c = num_people_c/num_visitors

print(num_people_c)
print(percent_visitors_c)

200.40080160320642
0.040096198800161346


Note that you need a smaller percentage of purchases for higher price points.

Now, for each group, perform a binomial test using `binom_test` from `scipy.stats`.
- `x` will be the number of purchases for that group
- `n` will be the total number of visitors assigned to that group
- `p` will be the target percent of purchases for that price point (calculated above)

Recall that:
- Group `A` is the \$0.99 price point
- Group `B` is the \$1.99 price point
- Group `C` is the \$4.99 price point

In [21]:
# import the binomial test from scipy.stats here
from scipy.stats import binom_test

In [23]:
# Test group A here
n = num_visitors / 3
print (n)
binom_test(purchase_counts['A']['Purchase'], n, percent_visitors_a)

1666.0


0.2111287299402726

In [24]:
# Test group B here
binom_test(purchase_counts['B']['Purchase'], n, percent_visitors_b)

0.20660209246555486

In [25]:
# Test group C here
binom_test(purchase_counts['C']['Purchase'], n, percent_visitors_c)

0.045623672477172125

If any of the groups passed the binomial test with $p < 0.05$, then we can be confident that enough people will buy the upgrade package at that price point to justify the feature.

Which price point should Brian go with?  Did this surprise you?

Brian should go with the price of $4.99 as the p value is less than .05. It means that we're confident that 95 percent of people will buy the microtransaction at price Point C. I was surprised since I was focus on the total number of people who bought it but after statistical analysis I was wrong. This is definitely an interesting project! 